In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from xgboost import XGBClassifier, plot_importance
import lightgbm as lgb
from lightgbm import LGBMClassifier, plot_importance, plot_tree
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler

import warnings
from tqdm import tqdm

import itertools
warnings.filterwarnings('ignore')

# 데이터 불러오기

In [2]:
train=pd.read_csv('data/train.csv')
test=pd.read_csv('data/test.csv')

# 파생변수 생성 및 인코딩

In [3]:
## 나이 양수로 변환
train['birth_plus'] = train.DAYS_BIRTH*(-1)/365
test['birth_plus'] = test.DAYS_BIRTH*(-1)/365

train['gender']=LabelEncoder().fit_transform(train.gender)
train['car']=LabelEncoder().fit_transform(train.car)
train['reality']=LabelEncoder().fit_transform(train.reality)
train['income_type']=LabelEncoder().fit_transform(train.income_type)
train['edu_type']=LabelEncoder().fit_transform(train.edu_type)
train['family_type']=LabelEncoder().fit_transform(train.family_type)
train['house_type']=LabelEncoder().fit_transform(train.house_type)
train.fillna('NAN',inplace=True)
train['occyp_type']=LabelEncoder().fit_transform(train.occyp_type)
train['income_weight']=train.income_total**2
train['income_age']=train.income_total*train.DAYS_BIRTH
train['income_emp']=train.income_total*train.DAYS_EMPLOYED

test['gender']=LabelEncoder().fit_transform(test.gender)
test['car']=LabelEncoder().fit_transform(test.car)
test['reality']=LabelEncoder().fit_transform(test.reality)
test['income_type']=LabelEncoder().fit_transform(test.income_type)
test['edu_type']=LabelEncoder().fit_transform(test.edu_type)
test['family_type']=LabelEncoder().fit_transform(test.family_type)
test['house_type']=LabelEncoder().fit_transform(test.house_type)
test.fillna('NAN',inplace=True)
test['occyp_type']=LabelEncoder().fit_transform(test.occyp_type)
test['income_weight']=test.income_total**2
test['income_age']=test.income_total*test.DAYS_BIRTH
test['income_emp']=test.income_total*test.DAYS_EMPLOYED

train.income_total=train.income_total.astype(int)
train.family_size=train.family_size.astype(int)
train.begin_month=train.begin_month.astype(int)
train.birth_plus=train.birth_plus.astype(int)

test.income_total=test.income_total.astype(int)
test.family_size=test.family_size.astype(int)
test.begin_month=test.begin_month.astype(int)
test.birth_plus=test.birth_plus.astype(int)

train.credit=train.credit.astype('category')
train.drop(['index','family_size','work_phone'],axis=1,inplace=True)
test.drop(['index','family_size','work_phone'],axis=1,inplace=True)
train.DAYS_BIRTH=train.DAYS_BIRTH**6
test.DAYS_BIRTH=test.DAYS_BIRTH**6
train.income_type=1/(train.income_type)
test.income_type=1/(test.income_type)
train.income_weight=1/(train.income_weight)**3
test.income_weight=1/(test.income_weight)**3
train['DAYS_BIRTH_weight']=1/(train.DAYS_BIRTH)**2
test['DAYS_BIRTH_weight']=1/(test.DAYS_BIRTH)**2

In [4]:
## predict_rf
ftr = train.drop('credit', axis=1).values
target = train['credit'].values
tst_ar = test.values
n_class = 3
n_fold = 17

In [5]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=40)

print(ftr.shape)

lgb_p_val = np.zeros((ftr.shape[0], n_class))
lgb_p_tst = np.zeros((tst_ar.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr, target), 1):
    print(f'training model for CV #{i}')
    lgb_clf = lgb.LGBMClassifier(max_depth=24,
                       num_leaves=110,
                       colsample_bytree=0.3,
                       n_estimators=175, 
                       min_child_samples=2,
                       subsample=0.9,
                       subsample_freq=2,
                       learning_rate=0.09,
                       random_state=2021,
                       verbose = 1)

    lgb_clf.fit(ftr[i_trn], target[i_trn],
            eval_set=[(ftr[i_val], target[i_val])],
            eval_metric='multi_error')

    lgb_p_val[i_val, :] = lgb_clf.predict_proba(ftr[i_val])
    lgb_p_tst += lgb_clf.predict_proba(tst_ar) / n_fold
    
print(f'{log_loss(target, lgb_p_val)}')
print(f'{confusion_matrix(target, np.argmax(lgb_p_val, axis=1))}%')

(26457, 21)
training model for CV #1
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000871 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1758
[LightGBM] [Info] Number of data points in the train set: 24900, number of used features: 20
[LightGBM] [Info] Start training from score -2.105645
[LightGBM] [Info] Start training from score -1.440254
[LightGBM] [Info] Start training from score -0.444156
[1]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.878837
[2]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.874335
[3]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.857246
[4]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.853795
[5]	valid_0's multi_error: 0.358382	valid_0's multi_logloss: 0.850344
[6]	valid_0's multi_error: 0.356455	valid_0's multi_logloss: 0.841431
[7]	valid_0's multi_error

[112]	valid_0's multi_error: 0.266538	valid_0's multi_logloss: 0.68894
[113]	valid_0's multi_error: 0.26718	valid_0's multi_logloss: 0.688566
[114]	valid_0's multi_error: 0.265896	valid_0's multi_logloss: 0.687684
[115]	valid_0's multi_error: 0.265896	valid_0's multi_logloss: 0.687608
[116]	valid_0's multi_error: 0.265896	valid_0's multi_logloss: 0.68718
[117]	valid_0's multi_error: 0.265896	valid_0's multi_logloss: 0.686822
[118]	valid_0's multi_error: 0.266538	valid_0's multi_logloss: 0.686616
[119]	valid_0's multi_error: 0.26718	valid_0's multi_logloss: 0.685912
[120]	valid_0's multi_error: 0.267823	valid_0's multi_logloss: 0.685736
[121]	valid_0's multi_error: 0.265896	valid_0's multi_logloss: 0.685574
[122]	valid_0's multi_error: 0.268465	valid_0's multi_logloss: 0.685304
[123]	valid_0's multi_error: 0.268465	valid_0's multi_logloss: 0.685265
[124]	valid_0's multi_error: 0.26718	valid_0's multi_logloss: 0.684515
[125]	valid_0's multi_error: 0.267823	valid_0's multi_logloss: 0.6846

[59]	valid_0's multi_error: 0.281952	valid_0's multi_logloss: 0.728337
[60]	valid_0's multi_error: 0.281952	valid_0's multi_logloss: 0.727808
[61]	valid_0's multi_error: 0.280026	valid_0's multi_logloss: 0.727707
[62]	valid_0's multi_error: 0.28131	valid_0's multi_logloss: 0.726751
[63]	valid_0's multi_error: 0.281952	valid_0's multi_logloss: 0.726139
[64]	valid_0's multi_error: 0.281952	valid_0's multi_logloss: 0.726007
[65]	valid_0's multi_error: 0.279383	valid_0's multi_logloss: 0.725151
[66]	valid_0's multi_error: 0.278741	valid_0's multi_logloss: 0.724564
[67]	valid_0's multi_error: 0.280668	valid_0's multi_logloss: 0.723656
[68]	valid_0's multi_error: 0.280668	valid_0's multi_logloss: 0.722906
[69]	valid_0's multi_error: 0.277457	valid_0's multi_logloss: 0.721664
[70]	valid_0's multi_error: 0.277457	valid_0's multi_logloss: 0.720048
[71]	valid_0's multi_error: 0.278099	valid_0's multi_logloss: 0.71932
[72]	valid_0's multi_error: 0.277457	valid_0's multi_logloss: 0.719297
[73]	val

training model for CV #3
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001007 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1760
[LightGBM] [Info] Number of data points in the train set: 24900, number of used features: 20
[LightGBM] [Info] Start training from score -2.105645
[LightGBM] [Info] Start training from score -1.440254
[LightGBM] [Info] Start training from score -0.444156
[1]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.876766
[2]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.872464
[3]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.859245
[4]	valid_0's multi_error: 0.359024	valid_0's multi_logloss: 0.855068
[5]	valid_0's multi_error: 0.358382	valid_0's multi_logloss: 0.852028
[6]	valid_0's multi_error: 0.358382	valid_0's multi_logloss: 0.844823
[7]	valid_0's multi_error: 0.358382	v

[116]	valid_0's multi_error: 0.268465	valid_0's multi_logloss: 0.691005
[117]	valid_0's multi_error: 0.26975	valid_0's multi_logloss: 0.690557
[118]	valid_0's multi_error: 0.269107	valid_0's multi_logloss: 0.690075
[119]	valid_0's multi_error: 0.269107	valid_0's multi_logloss: 0.68968
[120]	valid_0's multi_error: 0.269107	valid_0's multi_logloss: 0.689296
[121]	valid_0's multi_error: 0.26975	valid_0's multi_logloss: 0.689071
[122]	valid_0's multi_error: 0.269107	valid_0's multi_logloss: 0.688559
[123]	valid_0's multi_error: 0.26975	valid_0's multi_logloss: 0.688276
[124]	valid_0's multi_error: 0.26975	valid_0's multi_logloss: 0.688045
[125]	valid_0's multi_error: 0.26975	valid_0's multi_logloss: 0.687691
[126]	valid_0's multi_error: 0.269107	valid_0's multi_logloss: 0.687488
[127]	valid_0's multi_error: 0.269107	valid_0's multi_logloss: 0.686488
[128]	valid_0's multi_error: 0.267823	valid_0's multi_logloss: 0.685626
[129]	valid_0's multi_error: 0.266538	valid_0's multi_logloss: 0.68539

...

[34]	valid_0's multi_error: 0.298843	valid_0's multi_logloss: 0.755957
[35]	valid_0's multi_error: 0.293059	valid_0's multi_logloss: 0.752197
[36]	valid_0's multi_error: 0.293059	valid_0's multi_logloss: 0.751453
[37]	valid_0's multi_error: 0.291131	valid_0's multi_logloss: 0.749915
[38]	valid_0's multi_error: 0.294344	valid_0's multi_logloss: 0.748869
[39]	valid_0's multi_error: 0.291774	valid_0's multi_logloss: 0.747584
[40]	valid_0's multi_error: 0.291131	valid_0's multi_logloss: 0.744894
[41]	valid_0's multi_error: 0.289846	valid_0's multi_logloss: 0.74305
[42]	valid_0's multi_error: 0.289846	valid_0's multi_logloss: 0.742494
[43]	valid_0's multi_error: 0.28856	valid_0's multi_logloss: 0.741512
[44]	valid_0's multi_error: 0.287918	valid_0's multi_logloss: 0.740895
[45]	valid_0's multi_error: 0.287275	valid_0's multi_logloss: 0.74004
[46]	valid_0's multi_error: 0.287275	valid_0's multi_logloss: 0.738642
[47]	valid_0's multi_error: 0.284704	valid_0's multi_logloss: 0.736546
[48]	vali

[157]	valid_0's multi_error: 0.265424	valid_0's multi_logloss: 0.680846
[158]	valid_0's multi_error: 0.266067	valid_0's multi_logloss: 0.681294
[159]	valid_0's multi_error: 0.26671	valid_0's multi_logloss: 0.681518
[160]	valid_0's multi_error: 0.266067	valid_0's multi_logloss: 0.681617
[161]	valid_0's multi_error: 0.265424	valid_0's multi_logloss: 0.681084
[162]	valid_0's multi_error: 0.265424	valid_0's multi_logloss: 0.680935
[163]	valid_0's multi_error: 0.265424	valid_0's multi_logloss: 0.680716
[164]	valid_0's multi_error: 0.264139	valid_0's multi_logloss: 0.680627
[165]	valid_0's multi_error: 0.265424	valid_0's multi_logloss: 0.680924
[166]	valid_0's multi_error: 0.265424	valid_0's multi_logloss: 0.681029
[167]	valid_0's multi_error: 0.264781	valid_0's multi_logloss: 0.68111
[168]	valid_0's multi_error: 0.265424	valid_0's multi_logloss: 0.681238
[169]	valid_0's multi_error: 0.265424	valid_0's multi_logloss: 0.681051
[170]	valid_0's multi_error: 0.264781	valid_0's multi_logloss: 0.6

[96]	valid_0's multi_error: 0.280848	valid_0's multi_logloss: 0.704623
[97]	valid_0's multi_error: 0.279563	valid_0's multi_logloss: 0.704633
[98]	valid_0's multi_error: 0.279563	valid_0's multi_logloss: 0.704566
[99]	valid_0's multi_error: 0.278278	valid_0's multi_logloss: 0.703575
[100]	valid_0's multi_error: 0.27892	valid_0's multi_logloss: 0.702946
[101]	valid_0's multi_error: 0.278278	valid_0's multi_logloss: 0.702669
[102]	valid_0's multi_error: 0.27892	valid_0's multi_logloss: 0.702232
[103]	valid_0's multi_error: 0.27635	valid_0's multi_logloss: 0.701857
[104]	valid_0's multi_error: 0.27635	valid_0's multi_logloss: 0.701587
[105]	valid_0's multi_error: 0.277635	valid_0's multi_logloss: 0.701515
[106]	valid_0's multi_error: 0.276992	valid_0's multi_logloss: 0.701386
[107]	valid_0's multi_error: 0.275064	valid_0's multi_logloss: 0.701217
[108]	valid_0's multi_error: 0.275064	valid_0's multi_logloss: 0.700219
[109]	valid_0's multi_error: 0.275064	valid_0's multi_logloss: 0.700093


[36]	valid_0's multi_error: 0.298843	valid_0's multi_logloss: 0.759727
[37]	valid_0's multi_error: 0.300129	valid_0's multi_logloss: 0.758904
[38]	valid_0's multi_error: 0.299486	valid_0's multi_logloss: 0.756933
[39]	valid_0's multi_error: 0.299486	valid_0's multi_logloss: 0.755219
[40]	valid_0's multi_error: 0.296915	valid_0's multi_logloss: 0.752368
[41]	valid_0's multi_error: 0.294344	valid_0's multi_logloss: 0.750274
[42]	valid_0's multi_error: 0.294987	valid_0's multi_logloss: 0.749004
[43]	valid_0's multi_error: 0.294987	valid_0's multi_logloss: 0.747571
[44]	valid_0's multi_error: 0.294344	valid_0's multi_logloss: 0.746213
[45]	valid_0's multi_error: 0.294344	valid_0's multi_logloss: 0.745401
[46]	valid_0's multi_error: 0.294344	valid_0's multi_logloss: 0.744896
[47]	valid_0's multi_error: 0.291774	valid_0's multi_logloss: 0.742655
[48]	valid_0's multi_error: 0.291774	valid_0's multi_logloss: 0.741783
[49]	valid_0's multi_error: 0.293702	valid_0's multi_logloss: 0.739873
[50]	v

[159]	valid_0's multi_error: 0.272494	valid_0's multi_logloss: 0.704194
[160]	valid_0's multi_error: 0.271851	valid_0's multi_logloss: 0.704057
[161]	valid_0's multi_error: 0.270566	valid_0's multi_logloss: 0.703906
[162]	valid_0's multi_error: 0.269923	valid_0's multi_logloss: 0.704259
[163]	valid_0's multi_error: 0.269923	valid_0's multi_logloss: 0.704107
[164]	valid_0's multi_error: 0.268638	valid_0's multi_logloss: 0.703484
[165]	valid_0's multi_error: 0.26928	valid_0's multi_logloss: 0.703533
[166]	valid_0's multi_error: 0.269923	valid_0's multi_logloss: 0.703867
[167]	valid_0's multi_error: 0.270566	valid_0's multi_logloss: 0.703846
[168]	valid_0's multi_error: 0.269923	valid_0's multi_logloss: 0.704136
[169]	valid_0's multi_error: 0.271208	valid_0's multi_logloss: 0.704306
[170]	valid_0's multi_error: 0.272494	valid_0's multi_logloss: 0.704653
[171]	valid_0's multi_error: 0.271851	valid_0's multi_logloss: 0.704915
[172]	valid_0's multi_error: 0.269923	valid_0's multi_logloss: 0.

In [8]:
lgb_p_val

array([[0.12344968, 0.16451034, 0.71203998],
       [0.62961611, 0.04884707, 0.32153682],
       [0.03028669, 0.22971322, 0.74000009],
       ...,
       [0.05158067, 0.21021738, 0.73820195],
       [0.05503099, 0.15232557, 0.79264344],
       [0.04986553, 0.24448023, 0.70565424]])

In [62]:
print(f'{log_loss(target, lgb_p_val)}')
print(f'{confusion_matrix(target, np.argmax(lgb_p_val, axis=1))}%')

0.6811907370591721
[[  897   435  1890]
 [  221  2867  3179]
 [  414   782 15772]]%


----

In [66]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'lgb'
feature_name = '2021'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, lgb_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, lgb_p_tst, fmt='%.6f', delimiter=',')

In [ ]:
sample_submission=pd.read_csv('data/sample_submission.csv')
for i in range(17):
    sample_submission.iloc[:,1:]+=models_lgbm[i].predict_proba(test)/17
    
sample_submission.to_csv('state2021.csv',index=False)

In [29]:
feature_lgbm=feature_lgbm.sort_index()
feature_lgbm

print(np.mean(np.array(logloss_avg)))

0.6880214218100895


In [30]:
print(np.mean(np.array(logloss_avg)))

0.6880214218100895


- 90 : 0.6890796591713713

----

In [28]:
############################################################################
label=train.credit
train0=train.drop('credit',axis=1)
skf=StratifiedKFold(n_splits=17,random_state=40,shuffle=True)
folds=[]
for train_idx,val_idx in skf.split(train0,label):
    folds.append([train_idx,val_idx])
models_lgbm={}
logloss_avg=[]
feature_lgbm=pd.DataFrame()
for i in range(17):
    train_x,val_x,train_y,val_y=train0.iloc[folds[i][0],:],train0.iloc[folds[i][1],:],label[folds[i][0]],label[folds[i][1]]
    lgb=LGBMClassifier(max_depth=24,
                       num_leaves=110,
                       colsample_bytree=0.3,
                       n_estimators=230, 
                       min_child_samples=2,
                       subsample=0.9,
                       subsample_freq=2,
                       learning_rate=0.09,
                       random_state=99,
                       verbose = 50
                       # reg_alpha=0.1
                    )
    lgb.fit(train_x,train_y)
    logloss_avg.append(log_loss(val_y,lgb.predict_proba(val_x)))
    models_lgbm[i]=lgb
    feature_lgbm=pd.concat([feature_lgbm,pd.DataFrame(lgb.predict_proba(val_x),index=folds[i][1])],axis=0)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.748574
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.247245
[LightGBM] [Debug] init for col-wise cost 0.001282 seconds, init for row-wise cost 0.001238 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001496 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1758
[LightGBM] [Info] Number of data points in the train set: 24900, number of used features: 20
[LightGBM] [Debug] Use subset for bagging
[LightGBM] [Info] Start training from score -2.105645
[LightGBM] [Info] Start training from score -1.440254
[LightGBM] [Info] Start training from score -0.444156
[LightGBM] [Debug] Re-bagging, using 22369 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [De

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Re-bagging, using 22402 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Re-bagging, using 22422 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Deb

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22462 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Re-bagging, using 22376 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Deb

[LightGBM] [Debug] Re-bagging, using 22430 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Re-bagging, using 22393 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22406 data to train
[LightGBM] [Debug] Traine

[LightGBM] [Debug] Re-bagging, using 22414 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Re-bagging, using 22472 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Re-bagging, using 22369 data to train
[LightGBM] [Debug] Traine

...

[LightGBM] [Debug] Re-bagging, using 22451 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Re-bagging, using 22398 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22408 data to train
[LightGBM] [Debug] Traine

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22398 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22358 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Deb

[LightGBM] [Debug] Re-bagging, using 22420 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Re-bagging, using 22358 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22459 data to train
[LightGBM] [Debug] Traine

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22385 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Re-bagging, using 22443 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debu

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Re-bagging, using 22420 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22428 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Deb